# 구문 분석: Part V - PCFG

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 8.Analyzing sentence structure(https://www.nltk.org/book/ch08.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 8장 예제를 일부 사용.
- 참고 서적
    - Natural Language Processing with Python Cookbook, Krishna Bhavsar, Naresh Kumar, Pratap Dangeti, Packt Publishing, 2017.
    - Chapter 6, Chapter 7의 예제를 일부 사용.

### Prerequisite
동영상 강의(6장-하향식 구문분석 및 상향식 구문분석)을 시청한 후에 실습을 진행해야 함.

여기서 다룰 내용은
- 다양한 문장 구조를 표현하려면 형식 문법을 어떻게 사용하면 될까? 
- 구문 트리를 사용하여 문장 구조를 어떻게 보여줄 수 있을까?
- 파서가 문장을 어떻게 분석하고, 구문 트리를 자동으로 만들어 낼까?

In [ ]:
import nltk
from nltk.parse.generate import generate

## 3.5 확률 문법(Probabilistic CFG, PCFG)

각 생성규칙에 확률이 할당된 CFG. 
- Nontermial 기호에 대한 생성규칙의 확률 합은 1과 같아야 함. (**VP**의 경우 0.4+0.3+0.3=1.0)
- CFG가 생성하는 구문 트리와 같지만, 각 노드에 확률을 할당. 
- PCFG가 생성하는 노드의 확률은 해당 생성규칙과 이 생성규칙을 생성하는 규칙의 곱. 

In [ ]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

In [ ]:
print(grammar)

- PCFG 구문 분석기는 viterbi 알고리즘을 채택해 구현한 viterbi parser를 사용한다.
    - viterbi 알고리즘 관련 참고사이트 : https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html
        - viterbi 알고리즘은 HMM(hidden Markov Models) 문제를 확률 기반으로 해결. 
    - viterbi parser 참고사이트 : https://www.nltk.org/_modules/nltk/parse/viterbi.html
        - viterbi parser 알고리즘을 pseudo code로 설명
- **S -> NP VP**(1.0) x **NP -> 'Jack'** (0.2) x **VP -> TV NP** (0.4) x **TV -> 'saw'**(1.0) x **NP -> 'telescopes'**(0.8) = 0.064

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
    print(tree)
    tree.draw()

간단한 문법 예제를 통해 PCFG가 문장을 어떻게 생성하는지 알아보자.
- 이 문법은 길이가 1에서 4까지 단어를 생성한다.
- 첫번째 글자는 A이지만(확률 1), 두번째 글자는 B,C, 세번째 글자는 D,E,F, 마지막 글자는 G,H이다.

In [ ]:
productions = [
    "ROOT -> WORD [1.0]",
    "WORD -> P1 [0.25]",
    "WORD -> P1 P2 [0.25]",
    "WORD -> P1 P2 P3 [0.25]",
    "WORD -> P1 P2 P3 P4 [0.25]",
    "P1 -> 'A' [1.0]",
    "P2 -> 'B' [0.5]",    
    "P2 -> 'C' [0.5]",
    "P3 -> 'D' [0.3]",
    "P3 -> 'E' [0.3]",
    "P3 -> 'F' [0.4]",
    "P4 -> 'G' [0.9]",
    "P4 -> 'H' [0.1]",
]

In [ ]:
grammarString = "\n".join(productions)
grammar2 = nltk.PCFG.fromstring(grammarString)
print(grammar2)

주어진 문법이 어떤 문장을 생성하는지 모두 찾아보자. 
- **depth**는 구문 트리의 depth, **n**은 생성할 문장의 최대 갯수이다.
- n을 30으로 지정했지만 생성 가능한 문장 개수는 모두 21개이다. 
- 구문트리의 최대 depth는 3이지만, depth=**3+1**로 지정해야 한다.

In [ ]:
i = 0
for sent in generate(grammar2, n=30, depth=4):  
    wd = "".join(sent).replace(" ", "")
    print("{} >>> generated word: {}, size: {}".format(i, wd, len(wd)))   
    i += 1    

- 확률이 왜 0.005일까?
    - ROOT -> WORD [1.0]
    - WORD -> P1 P2 P3 P4 [0.25]
    - P1 -> 'A' [1.0], P2 -> 'C' [0.5], P3 -> 'F' [0.4], P4 -> 'H' [0.1]
    - 0.25x1.0x0.5x0.4x0.1=0.005

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar2)
for tree in viterbi_parser.parse(['A', 'C', 'F', 'H']):
    print(tree)
    tree.draw()